In [ ]:
# Imports
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
from pydantic import BaseModel
import json
import datetime
import your_information

# Webscraping 
Get job description text from the URL of the job posting using `Requests` and `BeautifulSoup`.

In [ ]:
# Send HTTP GET request
url_response = requests.get(your_information.job_description_url)

# Extract HTML content
html_content = url_response.text

# Initialize BeautifulSoup object to parse HTML content
soup = BeautifulSoup(html_content, "html.parser")

# Extract the text
job_description = soup.get_text()

# Remove whitespace characters
job_description = " ".join(job_description.split())  

# Show job description text
print(job_description)

# Information Extraction 
Get relevant information from job description text in JSON format using `OpenAI's chat completions API` with `response_format` parameter for `structured outputs`. Leverage the `Pydantic` library to define a clear and robust JSON schema for validating and structuring the extracted data.

In [ ]:
# Initialize OpenAI client to connect to ChatGPT API
client = OpenAI(api_key=your_information.OPENAI_API_KEY)

# Define JSON output format in English
class JobDescriptionExtraction(BaseModel):
    employer: str
    job_title: str
    requirements: list[str]
    tasks: list[str]
    contact_person: str
    address: str

# Define JSON output format in German
class StellenbeschreibungExtraktion(BaseModel):
    Arbeitgeber: str
    Stellenbezeichnung: str
    Anforderungen: list[str]
    Aufgaben: list[str]
    Kontaktperson: str
    Adresse: str

# Define system prompt for information extraction in English and German
extraction_system_prompt_dict = {
    "en": """
    You are an expert at structured data extraction. You will be given unstructured text from a job description 
    and should convert it into the given structure. If the information is missing in the job description, use 
    "unknown" as the value. Respond as concisely as possible.
    """,
    "de": """
    Du bist ein Experte für die Extraktion strukturierter Daten. Dir wird unstrukturierter Text aus einer 
    Stellenbeschreibung gegeben, den du in das vorgegebene Format umwandeln sollst. Wenn die Information 
    in der Stellenbeschreibung fehlt, verwende "unbekannt" als Wert. Antworte so kurz wie möglich.
    """
}

# Make API request to extract information from job description text
extraction_response_raw = client.beta.chat.completions.parse(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content": extraction_system_prompt_dict["de"]},  # "en" for English
        {"role": "user", "content": job_description}
    ],
    response_format = StellenbeschreibungExtraktion  # JobDescriptionExtraction for English
)

# Show raw extraction response
extraction_response_raw

In [ ]:
# Get dict from raw extraction response 
extraction_response_dict = extraction_response_raw.choices[0].message.parsed.dict()
extraction_response_dict

In [ ]:
# Convert dict to JSON  
extraction_response_json = json.dumps(extraction_response_dict, indent=4)
extraction_response_json

# Cover Letter Generation 
Create 3 cover letter suggestions by matching qualifications with job requirements using `OpenAI's chat completions API`.

In [ ]:
# Helper function: Get ChatGPT response from text prompt using API
def get_completion(prompt, system_prompt="", model="gpt-4o-mini"): 
    response = client.chat.completions.create(
        model = model,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    )
    return response

In [ ]:
# Cover letter prompt
cover_letter_prompt = f"""
Deine Aufgabe ist es, ein professionelles Bewerbungsanschreiben zu erstellen.

Adressiere das Anschreiben an folgenden Arbeitgeber, Adresse, Stelle und Kontaktperson:
Arbeitgeber: {extraction_response["Arbeitgeber"]}
Adresse: {extraction_response["Adresse"]}
Stelle: {extraction_response["Stellenbezeichnung"]}
Kontaktperson: {extraction_response["Kontaktperson"]}

Verwende folgenden Absender:
Name: {your_information.name}
Adresse: {your_information.address}
Telefonnummer: {your_information.phone}
E-Mail: {your_information.email}

Verwende im Briefkopf des Anschreibens Ort und Datum. 
Verwende den Ort aus: {your_information.address}
Verwende das folgende, aktuelle Datum im deutschen Datumsformat: {datetime.date.today()}

Beschreibe inwiefern die Anforderungen und Aufgaben erfüllt werden durch die 
Ausbildung, Arbeitserfahrung, Kompetenzen und Motivation. 
Verwende hierfür die folgenden Informationen:
Anforderungen: {extraction_response["Anforderungen"]}
Aufgaben: {extraction_response["Aufgaben"]}
Ausbildung: {your_information.education}
Arbeitserfahrung: {your_information.work_experience}
Kompetenzen: {your_information.skills}
Motivation: {your_information.motivation}

Nenne folgende Gehaltsvorstellung und mögliches Eintrittsdatum:
Gehaltsvorstellung : {your_information.salary_expectations}
Mögliches Eintrittsdatum: {your_information.possible_start_date}

Schreibe in einem professionellen, präzisen und kompakten Ton.

Unterschreibe das Anschreiben als {your_information.name}.
"""

# Create 3 cover letter suggestions
cover_letter_ls = []
for i in range(3):
    # Generate a single cover letter via API request
    cover_letter_response = get_completion(cover_letter_prompt, temperature=0.7).choices[0].message.content
    # Remove ** symbols
    cover_letter_response = cover_letter_response.replace('**', '')
    # Append cover letter to list
    cover_letter_ls.append(cover_letter_response)

# Print cover letter suggestions
for i in range(len(cover_letter_ls)):
    print(f"Anschreiben {i+1}")
    print("=" * 5)
    print(cover_letter_ls[i])
    print("=" * 30)